## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [162]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [163]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [ ]:
# Use the describe function to gather some basic statistics

df.describe()

In [ ]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df.info()

In [164]:
# What three item categories had the most entries?
unique_categories_series = df["category"].value_counts()

top_category = unique_categories_series.idxmax()
unique_categories_series.head(3)
#print(f"Category '{top_category}' had most entries")

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [165]:
# For the category with the most entries, which subcategory had the most entries?
consumables_df = df.loc[df["category"]==top_category]
subcategory_series = consumables_df["subcategory"].value_counts()
subcategory_series.dtype
top_sub_category = subcategory_series.idxmax()
print(f"Sub Category '{top_sub_category}' had most entries")

Sub Category 'bathroom supplies' had most entries


In [166]:
# Which five clients had the most entries in the data?
clientId_series = df["client_id"].value_counts()
clientId_series.head(5)

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [167]:
# Store the client ids of those top 5 clients in a list.
clientId_list = clientId_series.head(5).index.tolist()
clientId_list

[33615, 66037, 46820, 38378, 24741]

In [168]:
# How many total units (the qty column) did the client with the most entries order order?
clientId_list[0]
top_client_df = df[df["client_id"]==clientId_list[0]]
top_client_df["qty"].sum()


64313

In [ ]:
df.head()

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [169]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df["subtotal"] = df["unit_price"] * df["qty"]
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number,subtotal
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1,115164.00
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0,523.95
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6,527.28
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3,1056.18
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1,3902.00


In [136]:
#wgt_50 = df.loc[df['unit_weight']<50].copy()
# df['total_weight'] = df['unit_weight'] * df['qty']
# df.sort_values(by=["total_weight"], ascending=False)

In [177]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.

def calc_shipping_price(row):
    total_weight = row['unit_weight'] * row['qty']
    if (total_weight>50):
        return 7 * total_weight
    else:
        return 10 * total_weight
    
df["shipping_price"] = df.apply(calc_shipping_price, axis = 1)
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,subtotal,shipping_price,total_price,line_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,5512.50,125874.026250,80137.05,45736.976250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,312.90,588.693625,331.79,256.903625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,458.64,588.901200,318.30,270.601200
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,356.70,1167.314400,732.95,434.364400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,6500.20,4618.008425,2488.41,2129.598425


In [178]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
df["total_price"] = df.apply(lambda x: (x['subtotal']+x['shipping_price']) * (100 + 9.25)/100, axis = 1)
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,subtotal,shipping_price,total_price,line_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,5512.50,131839.076250,80137.05,45736.976250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,312.90,914.258625,331.79,256.903625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,458.64,1077.117600,318.30,270.601200
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,356.70,1543.571400,732.95,434.364400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,6500.20,11364.403500,2488.41,2129.598425


In [179]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).

df["line_cost"] = df.apply(lambda x: (x['unit_cost']* x['qty']) + x['shipping_price'], axis = 1)
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,subtotal,shipping_price,total_price,line_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,5512.50,131839.076250,85597.05,45736.976250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,312.90,914.258625,629.79,256.903625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,458.64,1077.117600,765.18,270.601200
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,356.70,1543.571400,1077.35,434.364400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,6500.20,11364.403500,8663.60,2129.598425


In [180]:
# Create a column for the profit of each line using line cost and line price
df["profit"] = df.apply(lambda x: (x['total_price'] - x['line_cost']) , axis = 1)
df.head()


,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,subtotal,shipping_price,total_price,line_cost,profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,5512.50,131839.076250,85597.05,46242.026250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,312.90,914.258625,629.79,284.468625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,458.64,1077.117600,765.18,311.937600
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,356.70,1543.571400,1077.35,466.221400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,6500.20,11364.403500,8663.60,2700.803500


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [181]:
# Check your work using the totals above
order_2742071 = df.loc[df['order_id']==2742071]
print(order_2742071['total_price'].sum())

order_2173913 = df.loc[df['order_id']==2173913]
print(order_2173913['total_price'].sum())

order_6128929 = df.loc[df['order_id']==6128929]
print(order_6128929['total_price'].sum())


152811.897075
162388.71929999997
923441.24045


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [182]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
for client in clientId_list:
    spend_df =  df.loc[df['client_id']==client]
    print (f"CLient: {client} Spend {round(spend_df['total_price'].sum(),2):.2f}")


CLient: 33615 Spend 8377308.57
CLient: 66037 Spend 10259514.80
CLient: 46820 Spend 9743794.32
CLient: 38378 Spend 12906550.88
CLient: 24741 Spend 82268891.98


In [185]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
df_dictionary = []
for client in clientId_list:
    spend_df =  df.loc[df['client_id']==client]
    total_units_purchased = spend_df['qty'].sum()
    total_shipping_price = spend_df['shipping_price'].sum()
    total_revenue = spend_df['total_price'].sum()
    total_profit = spend_df['profit'].sum()
    df_dictionary.append({
        "client_id":client,
        "total_units_purchased":total_units_purchased,
        "total_shipping_price":total_shipping_price,
        "total_revenue":total_revenue,
        "total_profit":total_profit
    })

summary_df = pd.DataFrame(df_dictionary)
summary_df

,client_id,total_units_purchased,total_shipping_price,total_revenue,total_profit
0,33615,64313,1828984.89,8.377309e+06,2.201995e+06
1,66037,43018,1395151.85,1.025951e+07,3.255032e+06
2,46820,75768,1601448.84,9.743794e+06,2.736603e+06
3,38378,73667,3429455.40,1.290655e+07,3.271830e+06
4,24741,239862,5126448.37,8.226889e+07,3.657999e+07


In [144]:
# Format the data and rename the columns to names suitable for presentation.

# Define the money columns. 


# Define a function that converts a dollar amount to millions.


# Apply the currency_format_millions function to only the money columns. 


# Rename the columns to reflect the change in the money format. 




In [145]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
